In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sqlite3


In [30]:
#conn = sqlite3.connect('az_1exp_pairsmix_10yrs.db')
conn = sqlite3.connect('../baselines/baseline_1exp_pairsmix_10yrs.db')

In [31]:
# Let's see if we can figure out why the slewtime is larger in this compared to the baseline
q = 'select filter,note,slewtime from SummaryAllProps'
df = pd.read_sql(q, conn)

In [32]:
vals = df.values

In [33]:
np.unique(vals[:,1])

array(['', 'DD:290', 'DD:COSMOS', 'DD:ECDFS', 'DD:ELAISS1', 'DD:XMM-LSS',
       'DD:u,290', 'DD:u,COSMOS', 'DD:u,ECDFS', 'DD:u,ELAISS1',
       'DD:u,XMM-LSS', 'blob, gr, a', 'blob, gr, b', 'blob, iz, a',
       'blob, iz, b', 'blob, ri, a', 'blob, ri, b', 'blob, u', 'blob, y',
       'blob, z'], dtype=object)

In [34]:
# I think it might be the transition between blobs that in increasing the mean
# Let's check the mean slewtime for just b-parts of blobs
is_b = []
for i,note in enumerate(vals[:,1]):
    if (len(note) > 0) & (vals[i,0] == vals[i-1,0]):
        is_b.append(note[-1] == 'a')
    else:
        is_b.append(False)

In [36]:
np.mean(vals[is_b,2]), np.mean(vals[np.where(vals[:,1] == ''), 2])

(6.812635187711417, 8.021347118452907)

7.944387010783188

In [37]:
conns = [sqlite3.connect('az_1exp_pairsmix_10yrs.db'), sqlite3.connect('../baselines/baseline_1exp_pairsmix_10yrs.db')]

In [44]:
print('a-block mean slewtime, b-block mean slew, greedy mean slewtime')
for conn in conns:
    q = 'select filter,note,slewtime from SummaryAllProps'
    df = pd.read_sql(q, conn)
    vals = df.values
    is_b = []
    is_a = []
    for i,note in enumerate(vals[:,1]):
        if (len(note) > 0) & (vals[i,0] == vals[i-1,0]):
            is_a.append(note[-1] == 'a')
            is_b.append(note[-1] == 'b')
        else:
            is_a.append(False)
            is_b.append(False)
    print(np.mean(vals[is_a,2]), np.mean(vals[is_b,2]),np.mean(vals[np.where(vals[:,1] == ''), 2]))

a-block mean slewtime, b-block mean slew, greedy mean slewtime
7.944387010783188 6.380589813036348 7.972263000943505
6.812635187711417 6.768513693859035 8.021347118452907


In [47]:
# so it looks like the a-block means are 1.1s higher than before. Is it just the initial one though?
# Chop 
print('a-block mean slewtime, b-block mean slew, greedy mean slewtime')
for conn in conns:
    q = 'select filter,note,slewtime from SummaryAllProps'
    df = pd.read_sql(q, conn)
    vals = df.values
    is_b = []
    is_a = []
    for i,note in enumerate(vals[:,1]):
        if (len(note) > 0) & (vals[i,0] == vals[i-1,0]):
            if vals[i,1] == vals[i-1,1]:
                is_a.append(note[-1] == 'a')
                is_b.append(note[-1] == 'b')
            else:
                is_a.append(False)
                is_b.append(False)
        else:
            is_a.append(False)
            is_b.append(False)
    print(np.mean(vals[is_a,2]), np.mean(vals[is_b,2]),np.mean(vals[np.where(vals[:,1] == ''), 2]))

a-block mean slewtime, b-block mean slew, greedy mean slewtime
6.276311870029339 6.380589813036348 7.972263000943505
6.477574044993482 6.768513693859035 8.021347118452907


In [ ]:
# So yes, there is an issue that the slewtime of the initial a-block observation is increasing the mean slewtime of all a-block by around a second.

The issue could be

* Need to add a detailer to reorder the traveling salesman so it slews to the closest part of the new block
* Need to increase weight on the azimuth basis function
* maybe I messed up the math on the basis function and it's slewing all over the place on accident